In [4]:
DEPTH_OPTIONS      = [3, 5, 7, 9]               # number of dilated layers
LEARNING_RATES     = [1e-1, 1e-2, 1e-3, 1e-4]
BATCH_SIZES        = [32, 64, 128, 256]
DILATION_TEMPLATE  = [1, 2, 4, 8, 16, 32, 64]   # will slice to desired depth
EPOCHS             = 50   # per trial (adjust)
VAL_SPLIT          = 0.2  # inside train set

In [6]:
import os, glob, re, itertools, numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix)
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

FEATURE_DIR_SCD = "SCD_Features_CSV_17apr"  # ← change if needed
FEATURE_DIR_NSR = "NSR_Features_CSV_17apr"  # ← change if needed

In [8]:
FEATURE_DIR = "NSR_SCD_FEATS_EX"

In [7]:
def parse_filename(path):
    """Return (is_scd, subject_id, segment_idx) where segment_idx∈{0…5}.
    SCD:  'SCD_30_First_5_mins_before_SCD.csv'  → is_scd=1,  subj='30', idx=5
    NSR:  'NSR_16265_Second_5_min.csv'          → is_scd=0,  subj='16265', idx=1
    Mapping:  First→0 … Sixth→5  (chronological)"""
    name = os.path.basename(path)
    is_scd = name.startswith("SCD")
    parts  = name.split('_')
    subject = parts[1]
    # segment word is either 'First','Second',… or index in list
    segment_word = parts[2]
    order = ["First", "Second", "Third", "Fourth", "Fifth", "Sixth"]
    idx = order.index(segment_word)
    if is_scd:
        # reverse chronological for SCD (Sixth is earliest)
        idx = 5 - idx
    return int(is_scd), subject, idx

In [11]:
def load_segments(feature_dir):
    subject_map = {}
    for path in glob.glob(os.path.join(feature_dir, "*.csv")):
        is_scd, subj, seg_idx = parse_filename(path)
        label = is_scd
        arr = pd.read_csv(path).values.squeeze()
        if subj not in subject_map:
            subject_map[subj] = {"label": label, "segments": [None]*6}
        subject_map[subj]["segments"][seg_idx] = arr
    return subject_map

In [12]:
subject_dict = load_segments(FEATURE_DIR)

In [14]:
print("Loaded", len(subject_dict), "subjects")

Loaded 36 subjects


In [16]:
X_train, y_train, X_test, y_test = [], [], [], []

for subj, info in subject_dict.items():
    label = info["label"]
    segs  = info["segments"]
    first_seg  = segs[0]
    other_segs = segs[1:]

    X_train.append(first_seg)
    y_train.append(label)

    X_test.extend(other_segs)
    y_test.extend([label]*5)

In [20]:
X_train

[array([0.72310915, 0.37386428, 0.04552058, 0.27241548, 0.0376728 ,
        0.188     , 0.5078125 , 2.5       ]),
 array([0.69657494, 0.50386794, 0.0943662 , 0.36251678, 0.05204275,
        0.402     , 0.125     , 2.0390625 ]),
 array([0.72436286, 0.35770021, 0.0486618 , 0.26221834, 0.03619986,
        0.2       , 0.2578125 , 3.3828125 ]),
 array([0.68579774, 0.36172454, 0.06850575, 0.27994196, 0.0408199 ,
        0.298     , 0.140625  , 2.671875  ]),
 array([0.35287839, 0.26748531, 0.09410377, 0.16660332, 0.04721267,
        0.798     , 0.171875  , 1.390625  ]),
 array([0.38639575, 0.23886734, 0.08861578, 0.19915234, 0.05154103,
        0.685     , 0.125     , 2.2578125 ]),
 array([0.43352582, 0.3497166 , 0.09535559, 0.24108649, 0.05561064,
        0.657     , 0.1796875 , 2.6171875 ]),
 array([0.45689985, 0.35058608, 0.09617152, 0.23430945, 0.05128245,
        0.628     , 0.1875    , 2.5       ]),
 array([0.76884665, 0.6323237 , 0.09379845, 0.45377054, 0.05901964,
        0.363     , 

In [23]:
len(X_test)

180

In [24]:
X_train = np.vstack(X_train)
X_test  = np.vstack(X_test)

In [27]:
X_train

array([[7.23109149e-01, 3.73864278e-01, 4.55205811e-02, 2.72415481e-01,
        3.76728025e-02, 1.88000000e-01, 5.07812500e-01, 2.50000000e+00],
       [6.96574941e-01, 5.03867935e-01, 9.43661972e-02, 3.62516776e-01,
        5.20427529e-02, 4.02000000e-01, 1.25000000e-01, 2.03906250e+00],
       [7.24362864e-01, 3.57700208e-01, 4.86618005e-02, 2.62218336e-01,
        3.61998591e-02, 2.00000000e-01, 2.57812500e-01, 3.38281250e+00],
       [6.85797735e-01, 3.61724539e-01, 6.85057471e-02, 2.79941957e-01,
        4.08199010e-02, 2.98000000e-01, 1.40625000e-01, 2.67187500e+00],
       [3.52878386e-01, 2.67485308e-01, 9.41037736e-02, 1.66603322e-01,
        4.72126739e-02, 7.98000000e-01, 1.71875000e-01, 1.39062500e+00],
       [3.86395753e-01, 2.38867341e-01, 8.86157827e-02, 1.99152337e-01,
        5.15410265e-02, 6.85000000e-01, 1.25000000e-01, 2.25781250e+00],
       [4.33525815e-01, 3.49716595e-01, 9.53555878e-02, 2.41086486e-01,
        5.56106414e-02, 6.57000000e-01, 1.79687500e-01, 2.

In [28]:
# standardise on TRAIN, apply to TEST
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [29]:
X_train

array([[-6.26032307e-02, -3.43556065e-01, -2.55496657e+00,
        -3.17583773e-01, -4.40155979e-01, -1.13120539e+00,
         2.33262745e+00, -2.81735266e-01],
       [-1.17453510e-01, -2.68640156e-01,  7.44645912e-01,
        -2.57587611e-01, -2.60552276e-01,  1.28705890e-01,
        -1.13973949e+00, -3.11574984e-01],
       [-6.00116083e-02, -3.52870772e-01, -2.34277134e+00,
        -3.24373793e-01, -4.58565652e-01, -1.06055616e+00,
         6.49592455e-02, -2.24584619e-01],
       [-1.39731650e-01, -3.50551711e-01, -1.00227573e+00,
        -3.12572085e-01, -4.00821776e-01, -4.83587442e-01,
        -9.98010226e-01, -2.70608591e-01],
       [-8.27927046e-01, -4.04858011e-01,  7.26918708e-01,
        -3.88041402e-01, -3.20921314e-01,  2.46013050e+00,
        -7.14551700e-01, -3.53552892e-01],
       [-7.58641505e-01, -4.21349401e-01,  3.56194685e-01,
        -3.66367840e-01, -2.66823134e-01,  1.79485025e+00,
        -1.13973949e+00, -2.97413762e-01],
       [-6.61216431e-01, -3.574714

In [30]:
# pad dummy zero to length=9
X_train = np.pad(X_train, ((0,0),(0,1)))
X_test  = np.pad(X_test,  ((0,0),(0,1)))

In [31]:
X_train

array([[-6.26032307e-02, -3.43556065e-01, -2.55496657e+00,
        -3.17583773e-01, -4.40155979e-01, -1.13120539e+00,
         2.33262745e+00, -2.81735266e-01,  0.00000000e+00],
       [-1.17453510e-01, -2.68640156e-01,  7.44645912e-01,
        -2.57587611e-01, -2.60552276e-01,  1.28705890e-01,
        -1.13973949e+00, -3.11574984e-01,  0.00000000e+00],
       [-6.00116083e-02, -3.52870772e-01, -2.34277134e+00,
        -3.24373793e-01, -4.58565652e-01, -1.06055616e+00,
         6.49592455e-02, -2.24584619e-01,  0.00000000e+00],
       [-1.39731650e-01, -3.50551711e-01, -1.00227573e+00,
        -3.12572085e-01, -4.00821776e-01, -4.83587442e-01,
        -9.98010226e-01, -2.70608591e-01,  0.00000000e+00],
       [-8.27927046e-01, -4.04858011e-01,  7.26918708e-01,
        -3.88041402e-01, -3.20921314e-01,  2.46013050e+00,
        -7.14551700e-01, -3.53552892e-01,  0.00000000e+00],
       [-7.58641505e-01, -4.21349401e-01,  3.56194685e-01,
        -3.66367840e-01, -2.66823134e-01,  1.794850

In [32]:
def build_wavenet(depth, lr, batch_size):
    dilations = DILATION_TEMPLATE[:depth]
    inp = Input(shape=(9,))
    x   = layers.Reshape((9,1))(inp)
    skips = []
    for d in dilations:
        t = layers.Conv1D(N_FILTERS, KERNEL_SIZE, padding="causal",
                          dilation_rate=d, activation="tanh")(x)
        s = layers.Conv1D(N_FILTERS, KERNEL_SIZE, padding="causal",
                          dilation_rate=d, activation="sigmoid")(x)
        g = layers.Multiply()([t,s])
        skips.append(layers.TimeDistributed(layers.Dense(N_FILTERS,activation='relu'))(g))
        x = layers.Add()([x,g])
    x = layers.Add()(skips)
    x = layers.Activation('relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    x = layers.Dense(DENSE_UNITS,activation='relu')(x)
    x = layers.Dropout(DROPOUT_RATE)(x)
    out = layers.Dense(1,activation='sigmoid')(x)
    model = Model(inp,out)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                  loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [33]:
best_cfg, best_loss, best_acc = None, np.inf, 0
for depth, lr, bs in itertools.product(DEPTH_OPTIONS, LEARNING_RATES, BATCH_SIZES):
    model = build_wavenet(depth, lr, bs)
    history = model.fit(X_train, y_train,
                        epochs=EPOCHS,
                        batch_size=bs,
                        validation_split=VAL_SPLIT,
                        verbose=0)
    val_loss, val_acc = history.history['val_loss'][-1], history.history['val_accuracy'][-1]
    print(f"Depth {depth:2}, lr {lr:1.0e}, bs {bs:3} → val_loss {val_loss:.4f} acc {val_acc:.4f}")
    if val_loss < best_loss or (val_loss==best_loss and val_acc>best_acc):
        best_cfg, best_loss, best_acc = (depth, lr, bs), val_loss, val_acc
        best_model = model  # keep
print("BEST CONFIG → depth, lr, batch =", best_cfg, "val_loss", best_loss, "val_acc", best_acc)

NameError: name 'N_FILTERS' is not defined